<h1>Loop Runner - Single Xi</h1>

<b>Description:</b> Here, I'm evolving loops for the given simulation. I'm employing a new technique here of utilizing two different binned density matrices, one which is high resolution and localized on the halo, and a low resolution one which covers the rest of the box. I'm also shifting the coordinates so the halo is always in the center.

In [3]:
# Import Statements
import numpy as np
import scipy as sp
from scipy import integrate, interpolate
import scipy.fft as fft
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import h5py
import os
import time

In [26]:
# Global Rocket Parameters
xi = 50
Gmu = 1e-12
GammaFrac = 0.1
vf = 0.3

# Global Simulation Parameters
tmax = 100
zi = 127
h = 0.7
rhoScale = 4.78e-20 # Mpc / Msun

Lbox = 100
LLowRes, LHighRes = [100, 1]
binsLowRes, binsHighRes = [100, 100]
dxLowRes, dxHighRes = [LLowRes / binsLowRes, LHighRes / binsHighRes]

# Global Physical Parameters
t0 = 4213 / h # Mpc / h
H0 = 0.0003333

# No. of Rockets
densRocket = 1e-6 * ((Gmu * xi) ** (-3/2)) * (t0 ** -3) * ((1 + zi) ** 3)
nRockets = int(densRocket * ((LHighRes / 2) ** 3))

In [5]:
# Set the Environment
envi = 0
envs = ["Cluster", "Local"]
env = envs[envi]

if env == "Cluster":
    # Cluster dirs
    neilDir = "/cluster/tufts/hertzberglab/nshah14/"
    homeDir = "/cluster/tufts/hertzberglab/shared/Rockets/"
    simName = "Sim_100v"
    simDir = homeDir + simName + "/"
    Div = "/"
elif env == "Local":
    # Cluster dirs
    homeDir = "C:\\Users\\NeilShah\\Documents\\PhD\\Rocket Capture\\"
    simName = "Sim_100v"
    simDir = homeDir + simName + "\\"
    Div = "\\"

print("Working in the {} environment".format(env))

Working in the Local environment


Utility Functions

In [6]:
def getHubbleEvol():
    OMatter = 0.25
    OLambda = 0.75

    da_dt = lambda a, t: a * H0 * np.sqrt(OMatter / a**3 + OLambda)

    a0 = 1 / (1 + zi)
    tInt = np.linspace(0, 1.1 / H0, 1000)
    af = interpolate.InterpolatedUnivariateSpline(tInt, integrate.odeint(da_dt, y0=a0, t=tInt)[:, 0])
    aDotf = af.derivative(n=1)
    tEnd = sp.optimize.fsolve(lambda t: af(t) - 1.0, x0=(1.0 / H0))[0]

    tArr = np.asarray([(tEnd + t)**(t / tmax) - 1 for t in range(tmax)])
    aArr = np.asarray([af(t) for t in tArr])
    HArr = np.asarray([aDotf(t) / af(t) for t in tArr])

    return tArr, aArr, HArr

In [7]:
def getCoords(t):
    snapDir = simDir + "snapdir_{:03d}{}".format(t, Div)
    pathArr = np.asarray(os.listdir(snapDir))
    
    ptypeN = 3
    coordsArr = [np.empty((0, 3), dtype=float) for i in range(ptypeN)]

    for pi in range(ptypeN):
        ptype = pi + 1
        for pathi in np.arange(0, pathArr.size):
            datGet = "/PartType{:d}/Coordinates".format(ptype)
            try:
                coords = np.asarray(h5py.File(snapDir + pathArr[pathi], 'r')[datGet])
                coordsArr[pi] = np.concatenate([coordsArr[pi], coords], axis=0)
            except KeyError:
                # print("   (Warning! Could not find ptype {:d} for snapshot t = {:d})".format(ptype, t))
                pass
    
    return(coordsArr)

def getHaloCoords(t):
    haloPtype = 1

    groupDir = simDir + "groups_{:03d}{}".format(t, Div)
    snapDir = simDir + "snapdir_{:03d}{}".format(t, Div)
    groupPaths = np.asarray(os.listdir(groupDir))
    snapPaths = np.asarray(os.listdir(snapDir))

    coordsArr = getCoords(t)[haloPtype - 1]
    
    haloCoords = np.empty((0, 0))
    for i, pathi in enumerate(groupPaths):
        gfile = h5py.File(groupDir + pathi, 'r')
        try:
            haloInit = int(np.asarray(gfile["/Group/GroupOffsetType/"])[0, haloPtype - 1])
            haloN = int(np.asarray(gfile["/Group/GroupLen/"])[haloPtype - 1])
            haloCoords = coordsArr[haloInit:(haloInit + haloN + 1), :]
            break
        except KeyError:
            pass
    
    return(haloCoords)

def getHaloStartTime():
    pathSave = simDir + "HaloStartTime.npy"

    if os.path.exists(pathSave):
        tStart = np.load(pathSave)[0]
    else:
        tStart = 0
        go = True
        for t in range(tmax):
            groupDir = simDir + "groups_{:03d}{}".format(t, Div)
            snapDir = simDir + "snapdir_{:03d}{}".format(t, Div)
            groupPaths = np.asarray(os.listdir(groupDir))
            snapPaths = np.asarray(os.listdir(snapDir))
            
            for i, pathi in enumerate(groupPaths):
                gfile = h5py.File(groupDir + pathi, 'r')
                try:
                    catch = gfile["/Group/GroupOffsetType/"]
                    tStart = t
                    go = False
                    break
                except KeyError:
                    continue
            if not go:
                break
        np.save(pathSave, np.asarray([tStart]))
    
    return(tStart)

def getHaloC(t):
    haloCoords = getHaloCoords(t)
    if haloCoords.shape[0] == 0:
        t0 = getHaloStartTime()
        halo0 = getHaloCoords(t0)
        xc = np.asarray([np.mean(halo0[:, i]) for i in range(3)])
    else:
        xc = np.asarray([np.mean(haloCoords[:, i]) for i in range(3)])
    return(xc)

In [8]:
def getMass():
    tFix = 50
    massTab = np.zeros(3)

    snapDir = simDir + "snapdir_{:03d}/".format(tFix)
    path1 = snapDir + "snapshot_{:03d}.0.hdf5".format(tFix)
    fil = h5py.File(path1, 'r')

    ptypeN = 3
    for pi in np.arange(0, ptypeN):
        ptype = pi + 1
        datGet = "/PartType{:d}/Masses".format(ptype)
        massTab[pi] = np.asarray(fil[datGet])[0]

    return massTab

In [9]:
def getRhoxHC(t, L, bins, override=False):
    dx = L / bins
    xc = getHaloC(t)
    dirSave = simDir + "rhox_HaloCentered/rhox_HC__L_{:03d}_{:03d}__bins_{:04d}/".format(int(np.floor(L)), int(1e3 * np.mod(L, 1)), bins)
    if not os.path.exists(dirSave):
        os.makedirs(dirSave)
    pathSave = dirSave + "rhox_HC__L_{:03d}_{:03d}__bins_{:04d}__t_{:03d}.npy".format(int(np.floor(L)), int(1e3 * np.mod(L, 1)), bins, t)
    
    if os.path.exists(pathSave) and not override:
        rhox = np.load(pathSave)
    else:
        ptypeN = 3
        massArr = getMass()
        rhox = np.zeros((bins, bins, bins))
        coords = getCoords(t)
        for pi in np.arange(0, ptypeN):
            ptype = pi + 1
            mi = massArr[pi]

            for ci, coord in enumerate(coords[pi]):
                i, j, k = [int(np.floor((coord[i] - (xc[i] - L/2)) / dx)) for i in range(3)]
                try:
                    rhox[i, j, k] += mi
                except IndexError:
                    pass
        
        rhox *= rhoScale * 1e10
        np.save(pathSave, rhox)
    
    return(rhox)

def rhoxHCInstantiate():
    print("Instantiating Density Matrices:")
    LHighRes, LLowRes = [100, 1]
    binsHighRes, binsLowRes = [100, 100]

    T0 = time.time()
    for t in range(tmax):
        if np.mod(t, 10) == 0:
            print("   Processing: {:03d} / {:03d} (Time Elapsed: {}, Time Estimated Remaining: {})".format(t, tmax, toMS(time.time() - T0), toMS((tmax / (t + 1)) * (time.time() - T0))))
        catch1 = getRhoxHC(t, LHighRes, binsHighRes, override=False)
        catch2 = getRhoxHC(t, LLowRes, binsLowRes, override=False)
    print("Finished! Took {} total".format(toMS(time.time() - T0)), end="\n\n")

In [30]:
def getkArr(L, bins, override=False):
    kArrDir = simDir + "kArrs/"
    if not os.path.exists(kArrDir):
        os.makedirs(kArrDir)
    kArrPath = kArrDir + "kArr__L_{:03d}__bins_{:03d}.npy".format(L, bins)
    if os.path.exists(kArrPath) and not override:
        kArr = np.load(kArrPath)
        return(kArr)

    kArr = np.zeros((bins, bins, bins), dtype=float)
    halfBins = int(bins / 2)
    for i in range(bins):
        for j in range(bins):
            for k in range(bins):
                lx = - (4 * bins**2 / L**2) * np.sin((np.pi / bins) * (i - halfBins))**2
                ly = - (4 * bins**2 / L**2) * np.sin((np.pi / bins) * (j - halfBins))**2
                lz = - (4 * bins**2 / L**2) * np.sin((np.pi / bins) * (k - halfBins))**2
                if i == halfBins and j == halfBins and k == halfBins:
                    kArr[i, j, k] = 1
                else:
                    kArr[i, j, k] = lx + ly + lz
    
    np.save(kArrPath, kArr)
    return kArr

def getRhokLambda(t, L, bins):
    if L == LHighRes:
        if not 'kArrHighRes' in globals():
            global kArrHighRes
            kArrHighRes = getkArr(L, bins)
        kArr = kArrHighRes
        dxi = dxHighRes
    elif L == LLowRes:
        if not 'kArrLowRes' in globals():
            global kArrLowRes
            kArrLowRes = getkArr(L, bins)
        kArr = kArrLowRes
        dxi = dxLowRes
    else:
        print("ERROR! You are trying to get a kArr for an L value which does not equal either the High-Res or Low-Res L value")
        quit()
    
    rhox = getRhoxHC(t, L, bins) / (dxi**3)
    rhok = fft.ifftn(rhox)

    halfBins = int(bins / 2)
    rhokL = np.divide(rhok, kArr)
    rhokL[halfBins, halfBins, halfBins] = 0
    
    return(rhokL)

In [29]:
def getFg(t, pos, rhokLOverride=False):
    xC = np.asarray([Lbox / 2 for i in range(3)])
    if False in ((pos - xC) < LLowRes/2):
        return np.zeros(3)
    elif False in ((pos - xC) < LHighRes/2):
        x, y, z = [int(np.floor((pos[i] - (xC[i] - LLowRes/2)) / (dxLowRes))) for i in range(3)]
        Fmult = 4 * np.pi / (2 * dxLowRes)
        if (not 'rhokLLowRes' in globals()) or rhokLOverride:
            global rhokLLowRes
            rhokLLowRes = getRhokLambda(t, LLowRes, binsLowRes)
        rhokL = rhokLLowRes
    else:
        x, y, z = [int(np.floor((pos[i] - (xC[i] - LHighRes/2)) / (dxHighRes))) for i in range(3)]
        Fmult = 4 * np.pi / (2 * dxHighRes)
        if (not 'rhokLHighRes' in globals()) or rhokLOverride:
            global rhokLHighRes
            rhokLHighRes = getRhokLambda(t, LHighRes, binsHighRes)
        rhokL = rhokLHighRes
    
    phix = fft.fftn(rhokL).real
    try:
        Fg = Fmult * np.asarray([phix[x + 1, y, z] - phix[x - 1, y, z], phix[x, y + 1, z] - phix[x, y - 1, z], phix[x, y, z + 1] - phix[x, y, z - 1]])
    except IndexError:
        Fg = np.zeros(3)
    return(Fg)

In [12]:
def toMS(t):
    s = np.floor(np.mod(t, 60))
    m = np.floor(np.mod(t, 3600) / 60)
    h = np.floor(t / 3600)

    if t < 1:
        tstr = "{:f} s".format(t)
    elif t < 3600:
        tstr = "{:02d}m {:02d}s".format(int(m), int(s))
    else:
        tstr = "{}h {:02d}m {:02d}s".format(int(h), int(m), int(s))
    return tstr

In [ ]:
t0G = time.time()
print("Hello! Now Beginning Halo Centered Rocket Evolution")
print("This run has Global Parameters: xi = {}, {} Rockets, Gmu = {}, GammaFrac = {}, vf = {}".format(xi, nRockets, Gmu, GammaFrac, vf), end="\n\n")

print("Now Beginning Instantiation of rho matrices. Here I will calculate binned density matrices from the raw simulation data and save it to a file.")
rhoxHCInstantiate()

In [ ]:
print("\nNow Processing Rocket Evolution!:")

rocketSaveDir = simDir + "RocketTrajs_HC/"
if not os.path.exists(rocketSaveDir):
    os.makedirs(rocketSaveDir)
rocketSavePath = rocketSaveDir + "RT_HC__Run1__N_{:06d}__xi_{:03d}.npy".format(nRockets, xi)

xArr = np.zeros((nRockets, 3, tmax))
vArr = np.zeros((nRockets, 3, tmax))
FRocketArr = np.zeros((nRockets, 3))

xC = np.asarray([Lbox / 2 for i in range(3)])
xArr[:, :, 0] = np.asarray([xC + 0.5 * LHighRes * (2*np.random.rand(3)-1) for i in range(0, nRockets)])
vArr[:, :, 0] = 2.6 * (1 + zi) * np.sqrt(xi * Gmu) * vf * np.asarray([v / np.linalg.norm(v) for v in (2*np.random.rand(nRockets, 3)-1)])
FRocketArr = (H0 * GammaFrac / xi) * np.asarray([v / np.linalg.norm(v) for v in (2*np.random.rand(nRockets, 3)-1)])

tArr, aArr, HArr = getHubbleEvol()
dtArr = np.diff(tArr)
kArrLowRes = getkArr(LLowRes, binsLowRes, override=True)
kArrLowRes = getkArr(LHighRes, binsHighRes, override=True)

t0 = time.time()
for ti in np.arange(0, tmax - 1):
    t = tArr[ti]
    dti = dtArr[ti]
    ai = aArr[ti]
    Hi = HArr[ti]
    rhokLLowRes = getRhokLambda(ti, LLowRes, binsLowRes)
    rhokLHighRes = getRhokLambda(ti, LHighRes, binsHighRes)

    for ri in range(nRockets):
        FRocketi = FRocketArr[ri, :]
        Fgi = getFg(ti, xArr[ri, :, ti])
        vArr[ri, :, ti + 1] = vArr[ri, :, ti] - 2 * Hi * vArr[ri, :, ti] * dti + (dti / ai) * (Fgi + FRocketi)
        xArr[ri, :, ti + 1] = xArr[ri, :, ti] + dti * vArr[ri, :, ti]
    if np.mod(ti + 1, 10) == 0:
        print("   Processing t = {}/{} (Time Elapsed: {}, Estimated Time Remaining: {})".format(ti + 1, tmax, toMS(time.time() - t0), toMS((tmax / (ti + 1) - 1) * (time.time() - t0))))

print("Finished Processing Rocket Evolution! Time Taken: {}\n".format(toMS(time.time() - t0)))

np.save(rocketSavePath, np.asarray([xArr, vArr]))
print("Rocket Trajectories Successfully Exported. \nWe are now done, thank you! Total Time Taken: {}\n".format(toMS(time.time() - t0G)))